In [ ]:
import json
import pandas as pd
import plotly.express as px

import numpy as np

import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from flask_babel import gettext
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
import math
from datetime import datetime, date, timedelta


df_vaccines = pd.read_csv('static/csv/be-covid-vaccines.csv')

df_vaccines = df_vaccines.groupby(['DATE','REGION','DOSE']).agg({'COUNT':'sum'}).reset_index()
df_vaccines.columns = df_vaccines.columns.get_level_values(0)



# Doses Belgium

In [ ]:
def cumulated_doses(region):
    df = df_vaccines
    if region != 'All':
        df = df_vaccines[df_vaccines.REGION==region]
    fig = px.bar(df, x="DATE", y="COUNT", color="DOSE",color_discrete_map={'A':'#d7191c',
                                 'B':'#fdae61',
                                 'C':'#a6d96a',
                                 'E':'#1a9641'})

    # Edit the layout
    fig.update_layout(title="Total Daily Dose Injections in "+region,
                      xaxis_title='Date',
                      yaxis_title='Dose')
    fig.update_layout(template="plotly_white")
    return fig
cumulated_doses("All").show()
cumulated_doses("Brussels").show()
cumulated_doses("Wallonia").show()
cumulated_doses("Flanders").show()

In [ ]:
# source: statbel
pop = {
    "Belgium": 11492641 - 945889 - 989782,
    "Wallonia": 3645243 - 303612 - 317262,
    "Flanders": 6629143 - 526108 - 551110,
    "Brussels": 1218255 - 116169 - 121410
}

def time_remaining(region):
    df = df_vaccines
    if region != 'Belgium':
        df = df_vaccines[df_vaccines.REGION==region]
    else:
        df = df_vaccines.groupby(["DATE", "DOSE"]).sum().reset_index()
    df = df[df.DOSE == "A"].copy()
    df['AVG COUNT'] = df.rolling(7, center=True).mean()
    df['CUMSUM'] = df['COUNT'].cumsum()
    df['REMAINING'] = -df['CUMSUM'] + pop[region]
    df['TIME_NEEDED'] = df['REMAINING'] / df['AVG COUNT']
    fig = px.line(df[df.DATE > "2021-03-01"], x="DATE", y="TIME_NEEDED",
                  labels={
                         "DATE": "Date",
                         "TIME_NEEDED": "Days"
                     },
                    title=f"Days needed to vaccinate all the yet unvaccinated population (> 14y)<br>if the number of dose/day stays the same ({region})")
    return fig
time_remaining("Belgium").show()
time_remaining("Brussels").show()
time_remaining("Wallonia").show()
time_remaining("Flanders").show()